<img src="/static/base/images/logo.png?v=641991992878ee24c6f3826e81054a0f" alt="Jupyter Notebook">
<h1 style="text-align: center">FCA Analysis of Health Data</h1>

<h3>Prerequisites</h3>

- You must have Python 3 installed on your system (<a href="https://www.python.org/downloads/">Download</a>)
- You must have Jupyter installed on your system (<a href="https://jupyter.org/install">Download</a>)
- Some knowledge of Python may be required

<h3>Explanation of Notebook 5</h3>

In this notebook, you will analyse a simple health data set which will be used to derive a conceptual model.<br>
This will involve the combination of 2 or more data sets and run an FCA (Formal Concept Analysis) algorithm on it to form a lattice to describe its properties.

There will be three steps in total:

+ In the first step, you'll create a lattice using a simple ASCII-art style cross-table with object rows and property columns
+ In the second step, you'll do the same but with hospital data with the department or the postcode records
+ In the third step, you'll do the same again but this time with all the sheets in the hospital data combined into one sheet, to save the hassle, it is already merged into a file.

The hospital data will be in csv format and are listed below:

<code>Departments.csv</code> - Departments<br>
<code>Postcodes.csv</code> - Postcodes<br>
<code>AllData.csv</code> - All data in one sheet

They were initially in xlsx format (Excel) but was converted to csv with 1's replaced with X's.

<h3>Getting started</h3>

To get started with this notebook, you will need to first install two packages, one known as "concepts", and the other known as "Graphviz".<br>
You can do this using pip in Command Prompt (Windows), or the Terminal (MacOS/Linux):<br>
+ <code>pip install concepts</code>
+ <code>pip install graphviz</code>

Otherwise, run the cells below.

In [ ]:
pip install concepts

In [ ]:
pip install graphviz

Graphviz has an issue when in use regarding system variables, due to this, change the path of the following cell to the "bin" folder of the inside the graphviz's folder (If installed from Anaconda, this folder will be in the "pkgs" folder) after installing the package from above. 

This folder must contain some .DLL files, if not, check for another folder pressumably named "graphviz" inside the "bin" folder and set the path to that.

Also ensure that forward slashes are used and not backward slashes (This interferes with the path names).

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'

After doing so, you may proceed with the notebook.

<h3>CSV files</h3>

Since the csv files are fairly small, a view of each of them are included in here so that you wouldn't have to open it manually to view its data.

<h5>Departments.csv</h5>

|  Patient ID  | Emergency | Minor Injury      |     Urgent Care     |
|-----|--------------|------------------|--------------|
Patient 1|X|0|0
Patient 2|X|0|0
Patient 3|X|0|0
Patient 4|0|X|0
Patient 5|0|X|0
Patient 6|0|X|0
Patient 7|0|X|0
Patient 8|0|0|X
Patient 9|0|0|X
Patient 10|0|0|0

<h5>Postcodes.csv</h5>

|  Patient ID  | Code 1 | Code 2  | Code 3  | Code 4 | Code 5 | Code 6
|-----|--------------|------------------|--------------|---|----|----|
Patient 1|0|0|0|0|X|0
Patient 2|X|X|0|0|0|0
Patient 3|0|0|0|X|0|0
Patient 4|0|0|0|X|0|0
Patient 5|0|0|0|0|0|0
Patient 6|0|0|X|0|0|0
Patient 7|0|0|0|0|X|0
Patient 8|0|0|0|0|0|X
Patient 9|0|0|X|0|0|0
Patient 10|0|0|0|0|0|X


<h5>AllData.csv</h5>

|  Patient ID  | Emergency | Minor Injury      |     Urgent Care     | Code 1 | Code 2  | Code 3  | Code 4 | Code 5 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6 | Code 6
|-------------|----|------------------|--------------|--------------|------------------|--------------|---|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
Patient 1|X|0|0|0|0|0|0|X|0|0|0|0|X|0|0|0|0|0|0|0|0|0|X|0|0|0|
Patient 2|X|0|0|X|X|0|0|0|0|0|0|0|X|0|X|X|X|0|0|X|0|0|X|X|0|0|
Patient 3|X|0|0|0|0|0|X|0|0|0|0|0|X|0|0|0|0|0|0|X|0|X|0|0|0|X|
Patient 4|0|X|0|0|0|0|X|0|0|0|X|0|0|0|0|0|X|0|X|0|0|0|0|0|0|0|
Patient 5|0|X|0|0|0|0|0|0|0|0|X|0|0|0|X|0|0|0|X|0|0|0|0|0|0|0|
Patient 6|0|X|0|0|0|X|0|0|0|0|0|0|X|0|0|0|0|0|0|0|X|0|0|X|0|X|
Patient 7|0|X|0|0|0|0|0|X|0|0|0|0|X|0|0|X|0|0|0|X|X|0|0|0|0|X|
Patient 8|0|0|X|0|0|0|0|0|X|X|0|0|0|0|X|0| |X|0|0|0|0|0|0|X|0|
Patient 9|0|0|X|0|0|X|0|0|0|X|0|0|0|0|X|0|0|0|0|0|0|0|0|X|X|0|
Patient 10|0|0|0|0|0|0|0|0|X|0|0|X|0|X|0|0|X|0|0|0|0|0|0|X|0|0|


<h3>First step</h3>

As mentioned, you will be using a simple ASCII-art style cross-table with object rows and property columns, and for this example, we will use King Arthur, Sir Robin and the holy grail as the objects while the properties will be human, knight, king and mysterious.

This ASCII-art style cross-table is also known as a formal context and defines which objects have a given property and vice versa which properties a given object has.

In [ ]:
from concepts import Context #Import the concepts package

#Build the table
c = Context.fromstring('''
            |human|knight|king |mysterious|
 King Arthur|  X  |  X   |  X  |          |
 Sir Robin  |  X  |  X   |     |          |
 holy grail |     |      |     |     X    |
 ''')
c  # doctest: +ELLIPSIS    

You can see that the table separates its values by a line, and uses 'X' to mark the properties (Top) for the objects (Left).<br>
Running the above cell will display an object saying that it has 3 objects within along with 4 properties.

In [ ]:
c.intension(['King Arthur', 'Sir Robin'])

The cell above will give the common properties for between King Arthur and Sir Robin using the "intension" function", which are 'human' and 'knight'.<br>
We can also get the common objects for the selected properties using the "extension" function, check the cell below.

In [ ]:
c.extension(['human', 'knight'])

In a way, you can think of both of them as the inverse of each other.

Similarly, you can also get the closest matching objects-properties pair of objects/properties as shown below.

In [ ]:
c['Sir Robin',]

This will display a pair where on the left you'll get the objects closest to the entered value ('Sir Robin' in this case) and the common properties to the right.<br>
You'll notice that it's inclusive, meaning that the object itself is also included as a close object

You can add multiple items, hence the comma, an example is shown below with 'Sir Robin' and 'holy grail'.

In [ ]:
c['Sir Robin', 'holy grail']

You'll notice that this time instead it doesn't include to properties to the right as it did before, this is because 'holy grail' only has one property; mysterious, and the other objects do not have this property, yet it still includes 'King Arthur' as a close object as 'Sir Robin' has common properties with it.

Just like with the intension and extension, you can also do an inversion with properties on the closest pairs as well as shown below.

In [ ]:
c['king',]

This will get all the objects with the property 'king' on the left, and to the right you'll see all the common properties of those objects, but since there's only one object, you can test it with the 'knight' property like shown below.

In [ ]:
c['knight',]

This time, it displays two objects, 'King Arthur' and 'Sir Robin' on the left, this is because they are both marked in the property 'knight'. Onto the right, you'll see their common properties, 'human' and 'knight'.

Try do the same but for the "mysterious" and "king" properties, what do you see?

<b>Double click for the solution</b>

<!--
c['mysterious', 'king']
-->

Similarly, we can also print all the closest object-properties pairs in the lattice like so in the following cell.

In [ ]:
for extent, intent in c.lattice: #For each extend and intent in the lattice
    print('%r %r' % (extent, intent))

Now we can make use of the 'Graphviz' package by making a visualisation of this lattice as shown below.

In [ ]:
c.lattice.graphviz()

You can see in this visualisation that it displays all the closest properties respectively to their objects; 'King Arthur' object is the only object that's marked as 'king' so they are close together, the same can be said for 'holy grail' and 'mysterious' as well.

One thing different is 'human' and 'knight' properties which are together and close to only 'Sir Robin' object even though 'King Arthur' object also shares those two properties, but this is because they are both connected on the same line whereas 'holy grail' object isn't.

<h3>Second step</h3>

Now that we've seen the use of the 'concepts' package and created a graphical visualiation of a lattice that we've created using the 'graphviz' package, we can now apply the same to the hospital data, for this example, we'll do it with department data as it's more descriptive.

In [ ]:
from concepts import Context #Import the concepts package

#Build the table
departments = Context.fromfile("data/Departments.csv", frmat="csv")

Running the cell above will build a table by first importing the csv file for the departments, notice how the parameter "frmat" is used for this.

We will go straight into printing all the closest object-properties pairs as shown below.

In [ ]:
for extent2, intent2 in departments.lattice: #For each extend and intent in the lattice
    print('%r %r' % (extent2, intent2))

From this, you can see the list of properties at the start, and you can see which patients fall in which property, you can also do this by filtering it as shown below.

In [ ]:
departments['Minor Injury',]

We will then visualise this like we did before.

In [ ]:
departments.lattice.graphviz()

This time, you can easily see that patient 10 is the only patient that does not fall under any of the properties.

Since this was for departments, we can also do the same for the postcodes as well.<br>
Try attempt this on your own on the cell below.

<b>Double click for the solution</b>

<!--
from concepts import Context #Import the concepts package

#Build the table
postcodes = Context.fromfile("data/Postcodes.csv", frmat="csv")

for extent3, intent3 in postcodes.lattice:
    print('%r %r' % (extent3, intent3))

print(postcodes['Code 6',])

postcodes.lattice.graphviz()
-->

You will notice the similarity between both the departments and the postcodes data; in both, there's one patient who does not fall under any property.

<h3>Third step</h3>

Now that you've applied a formal concept analysis of the data of departments and postcodes regarding patients and visualised it, your next task is to combine not only departments and postcodes but also the admission dates as well as the ages, and then perform the same analysis, this will help you pick out more differences and similarities in the given health data.

In [ ]:
from concepts import Context #Import the concepts package

#Build the table
all = Context.fromfile("data/AllData.csv", frmat="csv")

for extent3, intent3 in all.lattice: #For each extend and intent in the lattice
    print('%r %r' % (extent3, intent3))
    
all.lattice.graphviz()

This time, the lattice is much bigger, this is because it includes 4 seperate data sets; patient records, postcodes, ages, and dates.<br>
If you focus on the patients, you'll see that all of the points they connect to are a property that they are marked to, take patient 1 for an example; the patient is marked on 'Code 5', 'Fri: pm', and 'Sun: pm' properties, so even though it may all seem like a mess, when comparing this to the tabular version you'll see that this is much more readable.

<h3>The end</h3>

This concludes the end of notebook 5, you've analysed three separate sets of small data where at the end you had merged several different sets together to do a formal concept analysis on, this also assisted in visualising the lattice gained from the analysis where you were able to see the closest object-property pairs.

<h3>Bibliography</h3>

+ <a href="#First-step">ASCII-art style cross-table</a> by Sebastian Bank, 2013-2019 - Retrieved 16th of July, 2019, from <a href="https://concepts.readthedocs.io/en/stable/manual.html">https://concepts.readthedocs.io/en/stable/manual.html</a>.